In [3]:
import cv2
import numpy as np

# Abrir o vídeo
cam = cv2.VideoCapture(0)

# Ler o primeiro frame como referência de fundo
ret, frame_reference = cam.read()
if not ret:
    print("Erro ao carregar o vídeo")
    cam.release()
    exit()

frame_reference = cv2.resize(frame_reference, (0, 0), fx=1.15, fy=1.15)
frame_reference = cv2.flip(frame_reference, 1)
frame_reference_gray = cv2.cvtColor(frame_reference, cv2.COLOR_BGR2GRAY)
frame_reference_gray = cv2.GaussianBlur(frame_reference_gray, (31, 31), 0)

# Inicializar variáveis para histórico de pessoas detectadas
people_detected = []
person_id = 0
TIMEOUT = 100  # Número de frames até considerarmos que a pessoa saiu

# Função para calcular o histograma de cores de um ROI
def get_color_histogram(roi):
    hist = cv2.calcHist([roi], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    return cv2.normalize(hist, hist).flatten()

while True:
    ret, frame = cam.read()

    if not ret:
        break
    
    frame = cv2.flip(frame, 1)
    frame = cv2.resize(frame, (0, 0), fx=1.15, fy=1.15)
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_gray = cv2.GaussianBlur(frame_gray, (31, 31), 0)

    # Subtrair o frame de referência do frame atual
    frame_delta = cv2.absdiff(frame_reference_gray, frame_gray)
    _, thresh = cv2.threshold(frame_delta, 25, 255, cv2.THRESH_BINARY)
    thresh = cv2.dilate(thresh, None, iterations=2)

    # Encontrar contornos
    contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filtrar contornos pequenos e criar uma lista de áreas
    valid_contours = []
    areas = [cv2.contourArea(contour) for contour in contours if cv2.contourArea(contour) >= 500]  # Ignorar pequenas áreas

    # Se houver contornos válidos, processe
    if areas:
        max_area = max(areas)

        # Manter apenas contornos que são uma fração significativa do maior contorno
        for contour in contours:
            area = cv2.contourArea(contour)
            if area >= 500 and area > max_area * 0.3:  # Exemplo: contornos que são pelo menos 30% do maior
                valid_contours.append(contour)

    # Loop sobre os contornos filtrados
    for contour in valid_contours:
        # Extrair ROI (Região de Interesse) da pessoa detectada
        x, y, w, h = cv2.boundingRect(contour)
        roi = frame[y:y+h, x:x+w]
        
        # Calcular histograma de cores como uma característica visual
        person_hist = get_color_histogram(roi)
        
        # Verificar se a pessoa já foi detectada
        new_person = True
        for person in people_detected:
            dist = cv2.compareHist(person['hist'], person_hist, cv2.HISTCMP_CORREL)
            if dist > 0.8:  # Se o histograma for semelhante, consideramos a mesma pessoa
                person['last_seen'] = (x, y)
                person['missed_frames'] = 0  # Reseta contador quando pessoa é vista
                new_person = False
                break

        # Se for uma nova pessoa, adicionar ao histórico
        if new_person:
            person_id += 1
            people_detected.append({
                'id': person_id,
                'hist': person_hist,
                'last_seen': (x, y),
                'missed_frames': 0  # Contador de frames sem detecção
            })
            cv2.putText(frame, f"Pessoa {person_id} detectada", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Desenhar a casca convexa ao redor da pessoa detectada
        hull = cv2.convexHull(contour)
        cv2.drawContours(frame, [hull], -1, (0, 255, 0), 2)

    # Atualiza o contador de frames perdidos para todas as pessoas
    for person in people_detected:
        if person['missed_frames'] > TIMEOUT:
            people_detected.remove(person)  # Remove pessoas que passaram do timeout
        else:
            person['missed_frames'] += 1  # Incrementa o contador se não foi detectada

    # Mostrar IDs de todas as pessoas detectadas recentemente
    for person in people_detected:
        if person['missed_frames'] <= TIMEOUT:
            cv2.putText(frame, f"Pessoa {person['id']}", person['last_seen'], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Mostrar o frame com as detecções
    cv2.imshow('Deteccao de Mudancas', frame)
    cv2.imshow('Thresh', thresh)
    cv2.imshow('Frame Delta', frame_delta)

    # Pressione 'q' para sair
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Liberar o vídeo e fechar as janelas
cam.release()
cv2.destroyAllWindows()
